# Group 4: Random Forest Model and Evaluation

In [59]:
# Initial imports
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline

# Needed for decision tree visualization
import pydotplus
from IPython.display import Image


## Loading and Preprocessing Sentiment Data

In [60]:
# Loading data
file_path = Path("..\Group4\sentiment_closing.csv")
df_sentiment_closing = pd.read_csv('sentiment_closing.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)
df_sentiment_closing.head()




,title_sent,text_sent,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,,
2022-02-04,0.0,1.0,0.137523,0.147592,0.137250,0.147503,0.147503,580740990
2022-02-05,-1.0,-1.0,0.147533,0.150140,0.145410,0.147141,0.147141,587009429
2022-02-06,-1.0,-1.0,0.147140,0.154775,0.145258,0.153770,0.153770,757004142
2022-02-07,-1.0,0.0,0.153803,0.171586,0.151741,0.165578,0.165578,1791358822
2022-02-08,0.0,0.0,0.165606,0.169145,0.153709,0.158405,0.158405,1190116000


In [61]:
# Define features set
X = df_sentiment_closing.copy()
X.drop(["title_sent","text_sent"], axis=1, inplace=True)
X.head()



,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-02-04,0.137523,0.147592,0.137250,0.147503,0.147503,580740990
2022-02-05,0.147533,0.150140,0.145410,0.147141,0.147141,587009429
2022-02-06,0.147140,0.154775,0.145258,0.153770,0.153770,757004142
2022-02-07,0.153803,0.171586,0.151741,0.165578,0.165578,1791358822
2022-02-08,0.165606,0.169145,0.153709,0.158405,0.158405,1190116000


In [62]:
# Define target vector
# y = df_sentiment_closing["title_compound"].ravel()
# y = df_sentiment_closing["title_compound"]
y = df_sentiment_closing[["title_sent", "text_sent"]].values.reshape(-1,1)
y[:5]



array([[ 0.],
       [ 1.],
       [-1.],
       [-1.],
       [-1.]])

In [63]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)



ValueError: Found input variables with inconsistent numbers of samples: [29, 58]

In [ ]:
# Creating StandardScaler instance
scaler = StandardScaler()



In [ ]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)



In [ ]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [ ]:
# Confirm scaled values
df_X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
round(df_X_train_scaled.mean(), 0)


In [ ]:
round(df_X_train_scaled.std(), 0)


## Fitting the Random Forest Model

In [ ]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)



Once the random forest model is created, it is fitted with the training data.

In [ ]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)


## Making Predictions Using the Random Forest Model

In [ ]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)


## Model Evaluation

In [ ]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)



In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))


## Feature Importance

In [ ]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf_model.feature_importances_



In [ ]:
# We can sort the features by their importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)



In [ ]:
# Visualize the features by importance
importances_df = pd.DataFrame(sorted(zip(rf_model.feature_importances_, X.columns), reverse=True))
importances_df.set_index(importances_df[1], inplace=True)
importances_df.drop(columns=1, inplace=True)
importances_df.rename(columns={0: 'Feature Importances'}, inplace=True)
importances_sorted = importances_df.sort_values(by='Feature Importances')
importances_sorted.plot(kind='barh', color='lightgreen', title= 'Features Importances', legend=False)

